<a href="https://colab.research.google.com/github/AldianYohanes/altarlink/blob/main/NotulaX_535230139.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NotulaX
Aplikasi

In [1]:
# @title

# Instalasi Library
!pip install torch transformers streamlit soundfile librosa fastapi uvicorn pydub python-dotenv tf-keras python-multipart
!pip install accelerate # Direkomendasikan untuk performa model Transformers
!pip install pyngrok # Pastikan pyngrok terinstal untuk ngrok

# Hapus folder lama jika ada (optional, tapi disarankan untuk kebersihan)
!rm -rf indonlg
!rm -rf indobenchmark-toolkit # Hapus folder toolkit lama jika ada

# KLONING REPOSITORI INDOBENCHMARK-TOOLKIT YANG MENGANDUNG IndoNLGTokenizer
!git clone https://github.com/indobenchmark/indobenchmark-toolkit.git /content/indobenchmark-toolkit

# Anda tidak perlu lagi masuk ke direktori ini atau menginstal requirements.txt dari sini secara langsung.
# Karena toolkit ini akan kita akses sebagai modul Python langsung.

# Instalasi ffmpeg (penting untuk pydub dan Whisper)
!apt update
!apt install ffmpeg -y

print("Semua dependensi dan repositori telah diinstal.")

Cloning into '/content/indobenchmark-toolkit'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 171 (delta 16), reused 12 (delta 12), pack-reused 152 (from 1)
Receiving objects: 100% (171/171), 706.54 KiB | 2.24 MiB/s, done.
Resolving deltas: 100% (82/82), done.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InR

In [2]:
import os

init_file_path = '/content/indobenchmark-toolkit/src/indobenchmark/__init__.py'
print(f"Mengubah file: {init_file_path}")

try:
    # Baca konten asli
    with open(init_file_path, 'r') as f:
        content = f.read()

    # Baris yang ingin kita tambahkan/pastikan ada
    import_line = "from . import tokenization_indonlg"

    # Periksa apakah baris sudah ada atau dikomentari
    if import_line not in content and f"# {import_line}" not in content:
        # Jika belum ada sama sekali atau dikomentari dengan cara lain, tambahkan di akhir
        new_content = content + f"\n{import_line}\n"
    elif f"# {import_line}" in content:
        # Jika dikomentari, hilangkan komentarnya
        new_content = content.replace(f"# {import_line}", import_line)
    elif "# from . import tokenization" in content:
        # Jika ada komentar lama dari `tokenization` (nama file sebelumnya)
        new_content = content.replace("# from . import tokenization", import_line)
        if import_line not in new_content: # Pastikan sudah ditambahkan
            new_content += f"\n{import_line}\n"
    else:
        # Jika sudah ada dan tidak dikomentari (tidak mungkin sampai sini, tapi untuk jaga-jaga)
        new_content = content

    # Tulis kembali konten ke file
    with open(init_file_path, 'w') as f:
        f.write(new_content)

    print(f"Konten '{init_file_path}' setelah perubahan:")
    !cat {init_file_path}
    print("\nModifikasi __init__.py selesai.")

except Exception as e:
    print(f"ERROR: Gagal memodifikasi __init__.py: {e}")
    print("Pastikan folder dan file ada di jalur yang benar.")

Mengubah file: /content/indobenchmark-toolkit/src/indobenchmark/__init__.py
Konten '/content/indobenchmark-toolkit/src/indobenchmark/__init__.py' setelah perubahan:
from .version import __version__
from .tokenization_indonlg import IndoNLGTokenizer
from . import tokenization_indonlg

Modifikasi __init__.py selesai.


In [3]:
%%writefile main_backend.py
# main_backend.py content
# DEBUG: Backend main_backend.py - Versi: 20240618_TranscriptionCleaning

# Pastikan import di paling atas
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import soundfile as sf
import os
import io
from pydub import AudioSegment
import numpy as np
import sys
import gc # Import garbage collector
import warnings
import re # Tambahkan ini untuk ekspresi reguler

# --- START: Debugging Imports ---
print("DEBUG: main_backend.py sedang dimuat...")
# --- END: Debugging Imports ---

# Suppress warnings from huggingface_hub for cleaner output
warnings.filterwarnings("ignore", category=UserWarning, module='huggingface_hub.utils._auth')
warnings.filterwarnings("ignore", category=FutureWarning, module='huggingface_hub.file_download')

_indobenchmark_toolkit_path = '/content/indobenchmark-toolkit/src'
if _indobenchmark_toolkit_path not in sys.path:
    sys.path.insert(0, _indobenchmark_toolkit_path)
    print(f"DEBUG: Ditambahkan '{_indobenchmark_toolkit_path}' ke sys.path untuk menemukan modul IndoNLGTokenizer.")

IndoNLGTokenizer = None
summarization_enabled = False # Flag untuk mengontrol apakah peringkasan dapat dilakukan

# --- START: Debugging IndoNLGTokenizer Import ---
print("DEBUG: Mencoba mengimpor IndoNLGTokenizer...")
# --- END: Debugging IndoNLGTokenizer Import ---

try:
    # Coba impor menggunakan jalur absolut yang kita tambahkan
    from indobenchmark.tokenization_indonlg import IndoNLGTokenizer
    print("DEBUG: IndoNLGTokenizer berhasil diimpor secara eksplisit dari 'indobenchmark.tokenization_indonlg'.")
    summarization_enabled = True
except ImportError as e:
    # Jika gagal, coba impor dari root indobenchmark
    print(f"Peringatan: Gagal mengimpor IndoNLGTokenizer secara eksplisit dari 'indobenchmark.tokenization_indonlg'. Mencoba import alternatif. Error: {e}")
    try:
        # Ini adalah fallback jika struktur folder indobenchmark berubah atau tidak seperti yang diharapkan
        from tokenization_indonlg import IndoNLGTokenizer
        print("DEBUG: IndoNLGTokenizer berhasil diimpor dari 'tokenization_indonlg' (alternatif).")
        summarization_enabled = True
    except ImportError as e_alt:
        print(f"Peringatan KRITIS: Gagal mengimpor IndoNLGTokenizer bahkan dari lokasi alternatif. Fitur peringkasan akan dinonaktifkan. Error: {e_alt}")
        IndoNLGTokenizer = None # Pastikan IndoNLGTokenizer tidak terdefinisi
        summarization_enabled = False


app = FastAPI()

# Load Whisper model
try:
    print("DEBUG: Memuat model Whisper (whisper-base)...")
    whisper_pipe = pipeline("automatic-speech-recognition",
                            model="openai/whisper-large", # Dikembalikan ke whisper-base
                            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                            device=0 if torch.cuda.is_available() else -1,
                            generate_kwargs={"language": "id"}
                            )
    print("DEBUG: Model Whisper berhasil dimuat.")
    gc.collect() # Bersihkan memori setelah model Whisper dimuat
    torch.cuda.empty_cache() # Kosongkan cache GPU jika ada
except Exception as e:
    print(f"ERROR: Gagal memuat model Whisper: {e}.")
    print("Saran: Pastikan Anda memiliki GPU yang cukup atau coba model yang lebih kecil (misal 'whisper-tiny').")
    raise RuntimeError(f"Gagal memuat model Whisper: {e}")


# Load IndoBART for summarization
tokenizer = None
model_indobart = None

# Model IndoBART hanya akan dimuat jika IndoNLGTokenizer berhasil diimpor
if summarization_enabled and IndoNLGTokenizer is not None:
    try:
        print("DEBUG: Memuat tokenizer IndoBART...")
        tokenizer = IndoNLGTokenizer.from_pretrained("indobenchmark/indobart-v2")
        print("DEBUG: Tokenizer IndoBART berhasil dimuat.")

        if tokenizer.pad_token_id is None:
            tokenizer.pad_token_id = tokenizer.eos_token_id
            print(f"DEBUG: tokenizer.pad_token_id disetel ke eos_token_id: {tokenizer.pad_token_id}")

        print("DEBUG: Memuat model IndoBART...")
        model_indobart = AutoModelForSeq2SeqLM.from_pretrained("indobenchmark/indobart-v2")
        print("DEBUG: Model IndoBART berhasil dimuat.")

        model_indobart.eval()
        gc.collect() # Bersihkan memori setelah model IndoBART dimuat
        torch.cuda.empty_cache() # Kosongkan cache GPU jika ada

    except Exception as e:
        print(f"ERROR: Gagal memuat model atau tokenizer IndoBART: {e}")
        tokenizer = None
        model_indobart = None
        summarization_enabled = False # Nonaktifkan peringkasan jika ada masalah di sini
else:
    print("Peringatan: IndoNLGTokenizer tidak dimuat atau tidak diaktifkan, melewati pemuatan model IndoBART.")
    summarization_enabled = False # Pastikan ini false jika tokenizernya tidak ada


def clean_transcription(text: str) -> str:
    """
    Membersihkan transkripsi dari pengulangan kata atau pola yang berlebihan.
    Contoh: "mulu-mulu-mulu" -> "mulu-mulu", "kata kata kata kata" -> "kata kata"
    """
    # Mengurangi pengulangan dari 3+ menjadi 2 kali untuk pola 2-15 karakter
    # Ini akan menangani "mulu-mulu-mulu-mulu" menjadi "mulu-mulu"
    # Serta "duer duer duer duer" menjadi "duer duer"
    pattern_to_reduce = r'(.{2,15}?)(?:\s*|\W*)\1(?:(?:\s*|\W*)\1){1,}'
    # Penjelasan regex:
    # (.{2,15}?) : Kelompok tangkap 1. Menangkap pola 2 hingga 15 karakter, non-greedy.
    # (?:\s*|\W*) : Non-capturing group. Mencocokkan nol atau lebih spasi atau nol atau lebih non-word karakter.
    # \1          : Mencocokkan pengulangan pola yang sama dengan Kelompok tangkap 1.
    # (?:(?:\s*|\W*)\1){1,} : Mencocokkan 1 atau lebih pengulangan dari (spasi/non-word char + pola)
    # Ini berarti mencari 3 atau lebih pengulangan (pola pertama + 2 atau lebih pengulangan)

    # Fungsi pengganti untuk memastikan hanya ada dua kali pola yang diulang
    # Contoh: "mulu-mulu-mulu-mulu" -> "mulu-mulu"
    # "kata kata kata kata" -> "kata kata"
    def replacer(match):
        return match.group(1) + match.group(1) # Kembali ke dua kali pengulangan

    cleaned_text = re.sub(pattern_to_reduce, replacer, text, flags=re.IGNORECASE)

    # Menghapus pengulangan kata tunggal yang berlebihan (lebih dari 2 kali)
    # Contoh: "dan dan dan dan" -> "dan dan"
    cleaned_text = re.sub(r'\b(\w+)(?:\s+\1\b){2,}', r'\1 \1', cleaned_text, flags=re.IGNORECASE)

    return cleaned_text.strip()


@app.post("/transcribe_and_summarize/")
async def transcribe_and_summarize(audio_file: UploadFile = File(...)):
    if whisper_pipe is None: # Handle jika Whisper tidak dimuat (seharusnya tidak terjadi dengan raise RuntimeError di atas)
        raise HTTPException(status_code=500, detail="Model Transkripsi (Whisper) tidak dimuat. Silakan periksa log backend.")

    print(f"Menerima file: {audio_file.filename}, tipe: {audio_file.content_type}")
    if not audio_file.filename.endswith((".wav", ".mp3", ".ogg", ".flac")):
        raise HTTPException(status_code=400, detail="Format file audio tidak didukung. Harap unggah .wav, .mp3, .ogg, atau .flac.")

    full_transcribed_text = []
    transcribed_text = ""

    try:
        audio_bytes = await audio_file.read()
        print("DEBUG: Audio bytes berhasil dibaca.")

        audio = AudioSegment.from_file(io.BytesIO(audio_bytes))

        if audio.channels > 1:
            print(f"DEBUG: Mengonversi audio dari {audio.channels} kanal ke mono.")
            audio = audio.set_channels(1)

        if audio.frame_rate != 16000:
            print(f"DEBUG: Resampling audio dari {audio.frame_rate} Hz ke 16000 Hz.")
            audio = audio.set_frame_rate(16000)

        chunk_length_ms = 29 * 1000
        total_length_ms = len(audio)

        print(f"DEBUG: Total durasi audio: {total_length_ms / 1000:.2f} detik. Membagi menjadi chunk {chunk_length_ms / 1000:.2f} detik.")

        for i in range(0, total_length_ms, chunk_length_ms):
            chunk = audio[i:i + chunk_length_ms]

            buffer_chunk = io.BytesIO()
            chunk.export(buffer_chunk, format="wav")
            buffer_chunk.seek(0)

            audio_array_chunk, samplerate_chunk = sf.read(buffer_chunk)

            audio_array_chunk = audio_array_chunk.astype(np.float32)

            print(f"DEBUG: Memproses chunk {i/1000:.2f}s - {(i+len(chunk))/1000:.2f}s, bentuk: {audio_array_chunk.shape}")

            try:
                transcription_result = whisper_pipe(audio_array_chunk)
                full_transcribed_text.append(transcription_result["text"].strip())
                print(f"DEBUG: Chunk berhasil ditranskripsi. Teks awal: {transcription_result['text'][:50]}...")
            except Exception as e:
                print(f"ERROR: Gagal melakukan transkripsi pada chunk: {e}")
                full_transcribed_text.append(f"[Gagal transkripsi chunk {i/1000:.2f}s: {e}]")

        transcribed_text = " ".join(full_transcribed_text).strip()
        print(f"DEBUG: Transkripsi lengkap berhasil: {transcribed_text[:200]}...")

        # --- TAMBAHKAN PANGGILAN FUNGSI CLEANING DI SINI ---
        original_transcribed_text = transcribed_text # Simpan teks asli jika perlu untuk debugging
        transcribed_text = clean_transcription(transcribed_text)
        print(f"DEBUG: Transkripsi setelah dibersihkan: {transcribed_text[:200]}...")
        # --- AKHIR TAMBAHAN ---


    except Exception as e:
        print(f"ERROR: Gagal memproses file audio atau transkripsi secara keseluruhan: {e}")
        raise HTTPException(status_code=500, detail=f"Gagal memproses file audio atau transkripsi: {e}")

    summarized_text = "Tidak dapat meringkas teks. Model summarization belum tersedia atau bermasalah."

    inputs = None # Inisialisasi inputs di awal blok

    if summarization_enabled and tokenizer and model_indobart: # Cek flag summarization_enabled
        try:
            print("DEBUG: Memulai peringkasan dengan IndoBART (tokenisasi manual)...\n")

            max_input_length = model_indobart.config.max_position_embeddings
            print(f"DEBUG: Panjang teks transkripsi (karakter): {len(transcribed_text)}")
            print(f"DEBUG: Teks transkripsi awal: {transcribed_text[:500]}...\n")

            text_to_summarize = transcribed_text

            try:
                print("DEBUG: Melakukan tokenizer.tokenize...")
                tokens = tokenizer.tokenize(text_to_summarize)
                print(f"DEBUG: Tokenisasi berhasil. Jumlah token: {len(tokens)}")

                print("DEBUG: Melakukan tokenizer.convert_tokens_to_ids...")
                input_ids = tokenizer.convert_tokens_to_ids(tokens)
                print(f"DEBUG: Konversi ke ID berhasil. Jumlah ID: {len(input_ids)}")

                if tokenizer.bos_token_id is not None and tokenizer.eos_token_id is not None:
                    input_ids = [tokenizer.bos_token_id] + input_ids + [tokenizer.eos_token_id]
                    print(f"DEBUG: Token BOS/EOS ({tokenizer.bos_token_id}/{tokenizer.eos_token_id}) ditambahkan secara manual.")
                else:
                    print("Peringatan: tokenizer.bos_token_id atau eos_token_id tidak ditemukan. Lanjutkan tanpa menambahkan.")

                attention_mask = [1] * len(input_ids)

                print(f"DEBUG: Tokenisasi manual berhasil - panjang input_ids: {len(input_ids)}")

                if len(input_ids) > max_input_length:
                    print(f"Peringatan: Teks input terlalu panjang ({len(input_ids)} token). Memotong.")
                    input_ids = input_ids[:max_input_length]
                    attention_mask = attention_mask[:max_input_length]

                padding_length = max_input_length - len(input_ids)

                if padding_length > 0:
                    print(f"DEBUG: Menambahkan padding: {padding_length} token.")
                    input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
                    attention_mask = attention_mask + [0] * padding_length

                inputs = {
                    "input_ids": torch.tensor([input_ids], dtype=torch.long),
                    "attention_mask": torch.tensor([attention_mask], dtype=torch.long)
                }
                print(f"DEBUG: Ukuran input_ids untuk peringkasan (setelah manual padding): {inputs['input_ids'].shape}")

            except Exception as e_input_manual:
                print(f"ERROR: Gagal tokenisasi input atau padding manual untuk peringkasan: {e_input_manual}")
                summarized_text = f"Gagal meringkas: Kesalahan saat tokenisasi input. {e_input_manual}"
                raise e_input_manual # Lempar kembali exception


            if inputs is None:
                raise ValueError("Inputs is None after manual tokenization attempt, unexpected state.")

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model_indobart.to(device)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            print(f"DEBUG: Input tensor dipindahkan ke device: {device}")

            try:
                print("DEBUG: Menghasilkan ringkasan...")
                summary_ids = model_indobart.generate(
                    inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    max_length=150,
                    min_length=40,
                    num_beams=4,
                    do_sample=False,
                    no_repeat_ngram_size=2,
                    early_stopping=True
                )
                print(f"DEBUG: Ringkasan ID berhasil dihasilkan. Bentuk: {summary_ids.shape}")
            except Exception as e_generate:
                print(f"ERROR: Gagal saat generate ringkasan: {e_generate}")
                summarized_text = f"Gagal meringkas: Kesalahan saat menghasilkan ringkasan. {e_generate}"
                raise e_generate # Lempar kembali exception

            try:
                summarized_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
                print(f"DEBUG: Peringkasan berhasil: {summarized_text[:100]}...")
            except Exception as e_decode:
                print(f"ERROR: Gagal saat decode ringkasan: {e_decode}")
                summarized_text = f"Gagal meringkas: Kesalahan saat decode ringkasan. {e_decode}"
                raise e_decode # Lempar kembali exception

        except Exception as e:
            print(f"ERROR: Terjadi kesalahan umum dalam blok peringkasan IndoBART: {e}")
            summarized_text = f"Gagal meringkas teks dengan IndoBART (general error): {e}"
    else:
        print("Peringatan: Model peringkasan tidak tersedia karena IndoNLGTokenizer gagal dimuat atau dinonaktifkan.")
        summarized_text = "Model peringkasan tidak tersedia."

    return JSONResponse(content={"transcribed_text": transcribed_text, "summarized_text": summarized_text})


Overwriting main_backend.py


In [4]:
from pyngrok import ngrok, conf
import os
import time
import subprocess
import requests # Tambahkan ini untuk cek status backend

NGROK_AUTH_TOKEN = "2xozndD3skSORMuCWD8Yaa90eEO_3er9uWBw3H4zMXigtsEYQ" # Pastikan token Anda benar di sini
conf.get_default().auth_token = NGROK_AUTH_TOKEN

try:
    # Coba matikan semua proses ngrok yang mungkin berjalan
    subprocess.run(["killall", "ngrok"], check=True, capture_output=True, timeout=5)
    print("Existing ngrok processes killed using killall.")
except (subprocess.CalledProcessError, subprocess.TimeoutExpired) as e:
    if "no process found" in str(e):
        print("No ngrok processes found to kill.")
    else:
        print(f"Error killing ngrok processes (might not be running): {e}")
except FileNotFoundError:
    print("killall command not found, skipping. (This is normal in some environments)")

# Gunakan pyngrok.kill() sebagai fallback yang lebih halus
ngrok.kill()
print("pyngrok internal processes killed.")


# LANGKAH BARU: Jalankan FastAPI backend di background dan arahkan output ke file log
print("\nMemulai FastAPI backend di background dan mengarahkan log ke 'fastapi_backend.log'...")
with open('fastapi_backend.log', 'w') as log_file:
    backend_process = subprocess.Popen(
        ["python", "-m", "uvicorn", "main_backend:app", "--host", "0.0.0.0", "--port", "8000"],
        stdout=log_file, # Redirect stdout to file
        stderr=log_file, # Redirect stderr to file
        text=True,
        preexec_fn=os.setsid
    )
print("Proses FastAPI dimulai. Menunggu 10 detik agar server siap...")
time.sleep(10) # Beri waktu lebih agar FastAPI benar-benar memulai dan memuat model

# Cek apakah backend sudah listening
fastapi_ready = False
for _ in range(5): # Coba beberapa kali
    try:
        response = requests.get("http://localhost:8000/docs", timeout=5) # Coba akses endpoint /docs
        if response.status_code == 200:
            print("FastAPI backend berhasil merespons di localhost:8000!")
            fastapi_ready = True
            break
    except requests.exceptions.ConnectionError:
        print("Menunggu FastAPI backend...")
        time.sleep(2)
    except Exception as e:
        print(f"Error saat mengecek FastAPI: {e}")
        time.sleep(2)

# Setelah pengecekan selesai, cetak log dari file
print("\n--- Log Backend FastAPI (fastapi_backend.log) ---")
try:
    with open('fastapi_backend.log', 'r') as log_file:
        print(log_file.read())
except FileNotFoundError:
    print("File log 'fastapi_backend.log' tidak ditemukan.")
print("--- Akhir Log Backend FastAPI ---\n")


if not fastapi_ready:
    print("\nERROR KRITIS: FastAPI backend tidak merespons setelah beberapa upaya.")
    raise RuntimeError("FastAPI backend gagal memulai. Lihat log di atas untuk detail.")

print("\nFastAPI backend sekarang dianggap siap. Melanjutkan dengan ngrok untuk Streamlit saja.")

# HANYA Start ngrok tunnel untuk Streamlit port 8501
try:
    print("Starting ngrok tunnel for Streamlit (port 8501)...")
    public_url_streamlit_tunnel = ngrok.connect(8501, proto="http", name="streamlit_tunnel")
    public_url_streamlit = public_url_streamlit_tunnel.public_url
    print(f"Streamlit Public URL: {public_url_streamlit} -> http://localhost:8501")

except Exception as e:
    print(f"ERROR: Gagal memulai tunnel Streamlit: {e}")
    public_url_streamlit = None

# Verifikasi URL Streamlit
if not public_url_streamlit:
    print("Peringatan: URL Streamlit tidak ditemukan. Anda tidak akan dapat mengakses frontend.")


Error killing ngrok processes (might not be running): Command '['killall', 'ngrok']' returned non-zero exit status 1.
pyngrok internal processes killed.

Memulai FastAPI backend di background dan mengarahkan log ke 'fastapi_backend.log'...
Proses FastAPI dimulai. Menunggu 10 detik agar server siap...
FastAPI backend berhasil merespons di localhost:8000!

--- Log Backend FastAPI (fastapi_backend.log) ---
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [5]:
%%writefile main_frontend.py
import streamlit as st
import requests
import json
import base64
import os

# Konfigurasi halaman
st.set_page_config(
    page_title="NotulaX - Ringkasan Otomatis Notulen",
    page_icon="📝",
    layout="wide"
)

# Styling dengan CSS custom
st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Comfortaa:wght@400;700&display=swap');

    html, body, .stApp {
        background: url('https://drive.google.com/uc?export=view&id=1ftdEZ5jLxPbL2NK-7oXlNBBlcv9vCclB') no-repeat center center fixed;
        background-size: cover;
        font-family: 'Comfortaa', sans-serif;
        color: #1C1C1C; /* Warna teks default */
    }

    h1, h2, h3, h4, h5, h6, p {
        font-family: 'Comfortaa', sans-serif;
    }

    .upload-section {
        background-color: #952537; /* Merah Maroon lebih terang */
        padding: 40px 30px;
        border-radius: 30px;
        box-shadow: 0px 10px 25px rgba(0, 0, 0, 0.2);
        text-align: center;
        color: white; /* Teks di dalam kotak unggah akan putih */
    }

    .upload-section h2 {
        font-size: 1.8em;
        font-weight: 700;
        color: white; /* Pastikan judul dalam kotak putih */
    }

    .upload-section p {
        font-size: 1em;
        font-weight: 400;
        color: white; /* Pastikan paragraf dalam kotak putih */
    }

    /* Styling untuk st.file_uploader di dalam upload-section */
    .upload-section .stFileUploader > div:first-child > div {
        border: 2px dashed rgba(255, 255, 255, 0.7); /* Garis putus-putus */
        background-color: rgba(255, 255, 255, 0.05); /* Latar belakang transparan */
        color: white;
        padding: 30px;
        border-radius: 15px;
    }
    .upload-section .stFileUploader > div:first-child > div p {
        color: white; /* Teks "Drag and drop file here" */
    }
    .upload-section .stFileUploader label {
        visibility: hidden; /* Sembunyikan label default */
        height: 0;
        margin: 0;
    }

    /* Styling tombol "Browse files" dari st.file_uploader */
    .upload-section .stFileUploader button {
        background-color: #BDBDBD;
        color: #952537;
        font-weight: bold;
        border-radius: 50px;
        padding: 12px 40px;
        font-size: 1.1em;
        border: none;
        box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
        margin-top: 20px; /* Jarak dari area drop */
    }
    .upload-section .stFileUploader button:hover {
        background-color: #a5a5a5;
    }

    /* Styling tombol "Proses Notulen" dan lainnya */
    .stButton > button {
        background-color: #952537;
        color: white;
        font-weight: bold;
        border-radius: 50px;
        padding: 12px 40px;
        font-size: 1.1em;
        border: none;
        box-shadow: 0px 10px 25px rgba(0, 0, 0, 0.2);
    }

    .stButton > button:hover {
        background-color: #680f1d;
    }

    .logo-box {
        background-color: #595959;
        padding: 40px 30px;
        border-radius: 30px;
        color: white;
        text-align: center;
        box-shadow: 0px 10px 25px rgba(0, 0, 0, 0.2);
    }

    .logo-box img {
        width: 150px;
        margin-bottom: 20px;
    }

    .header-logo {
        font-size: 2.2em;
        font-weight: bold;
        color: #952537;
        font-family: 'Comfortaa', sans-serif; /* Pastikan Comfortaa */
    }

    /* Styling untuk output di dalam upload-section */
    .upload-section .stSubheader {
        color: white; /* Subheader hasil */
        margin-top: 30px;
        text-align: left; /* Sesuaikan alignment */
        font-size: 1.5em;
        font-weight: 700;
    }
    .upload-section .stTextArea label {
        color: #F0F0F0; /* Label area teks */
        text-align: left;
    }
    .upload-section .stTextArea div[data-baseweb="textarea"] textarea {
        background-color: rgba(255, 255, 255, 0.1);
        color: white;
        border: 1px solid rgba(255, 255, 255, 0.3);
        border-radius: 10px;
    }
    .upload-section .stDownloadButton > button {
        background-color: #BDBDBD; /* Warna unduh lebih terang */
        color: #952537;
        font-weight: bold;
        border-radius: 15px;
        padding: 8px 15px;
        margin-top: 10px;
        margin-right: 10px;
    }
    .upload-section .stDownloadButton > button:hover {
        background-color: #a5a5a5;
    }
    .upload-section .stInfo { /* Pesan info di dalam upload-section */
        background-color: rgba(255, 255, 255, 0.1);
        border-left: 5px solid rgba(255, 255, 255, 0.5);
        color: white;
        border-radius: 10px;
        padding: 15px;
        margin-top: 20px;
    }
    .upload-section .stInfo p {
        color: white !important;
    }
    .upload-section .stException { /* Pesan error di dalam upload-section */
        background-color: rgba(255, 0, 0, 0.2);
        border-left: 5px solid red;
        color: red;
        border-radius: 10px;
        padding: 15px;
        margin-top: 20px;
    }
    .upload-section .stException p {
        color: red !important;
    }
</style>
""", unsafe_allow_html=True)

# URL Backend
BACKEND_URL = "http://localhost:8000/transcribe_and_summarize/"

# Header
st.markdown("<h1 class='header-logo'>NotulaX</h1>", unsafe_allow_html=True)
st.markdown("### Ringkasan Notulen Otomatis - NLP Final Project by Aldian Yohanes", unsafe_allow_html=True)
st.markdown("---")

# Layout konten utama: kiri (unggah) dan kanan (logo/info)
left_col, right_col = st.columns([2.5, 1.5])

with left_col:
    # Menggunakan st.container() untuk membungkus konten upload-section
    # Ini penting agar CSS custom class 'upload-section' bisa diterapkan ke div ini
    with st.container():
        st.markdown('<div class="upload-section">', unsafe_allow_html=True)
        st.markdown("<h2>Unggah Filemu Disini</h2>", unsafe_allow_html=True)
        st.markdown("<p>Klik tombol 'Pilih File' untuk memulai atau drag and drop file ke area ini.</p>", unsafe_allow_html=True)

        uploaded_file = st.file_uploader("", type=["wav", "mp3", "ogg", "flac"], label_visibility="collapsed")

        # Logika pemrosesan audio dipindahkan ke sini
        if uploaded_file is not None:
            st.audio(uploaded_file, format=uploaded_file.type)
            if st.button("Proses Notulen"):
                with st.spinner("Sedang memproses audio... Ini mungkin memakan waktu beberapa menit tergantung ukuran file dan kecepatan server."):
                    try:
                        files = {"audio_file": (uploaded_file.name, uploaded_file.getvalue(), uploaded_file.type)}
                        response = requests.post(BACKEND_URL, files=files, timeout=900) # Timeout 15 menit

                        if response.status_code == 200:
                            result = response.json()
                            transcribed_text = result.get("transcribed_text", "Tidak ada transkripsi.")
                            summarized_text = result.get("summarized_text", "Tidak ada ringkasan.")

                            st.subheader("Hasil Transkripsi Rapat")
                            st.text_area("Transkripsi Lengkap", transcribed_text, height=300)

                            st.subheader("Notulen Rapat (Ringkasan)")
                            st.markdown(f"**Ringkasan:**\n{summarized_text}")
                            st.text_area("Notulen Ringkas", summarized_text, height=200)

                            # Fitur Unduh
                            st.subheader("Unduh Notulen")

                            # Menggunakan kolom untuk tombol unduh agar sejajar
                            download_col1, download_col2 = st.columns(2)
                            with download_col1:
                                st.download_button(
                                    label="Unduh Transkripsi (.txt)",
                                    data=transcribed_text.encode("utf-8"),
                                    file_name="transkripsi_rapat.txt",
                                    mime="text/plain",
                                    key="download_transcription"
                                )
                            with download_col2:
                                st.download_button(
                                    label="Unduh Ringkasan (.txt)",
                                    data=summarized_text.encode("utf-8"),
                                    file_name="ringkasan_rapat.txt",
                                    mime="text/plain",
                                    key="download_summary"
                                )

                        else:
                            st.error(f"Terjadi kesalahan pada server: {response.status_code} - {response.text}")
                            try:
                                st.json(response.json())
                            except json.JSONDecodeError:
                                st.write(response.text)
                    except requests.exceptions.Timeout:
                        st.error("Permintaan ke server memakan waktu terlalu lama. Silakan coba lagi atau gunakan file yang lebih kecil.")
                    except requests.exceptions.ConnectionError:
                        st.error(f"Tidak dapat terhubung ke server Back-End di {BACKEND_URL}. Pastikan Back-End berjalan dengan baik.")
                    except Exception as e:
                        st.error(f"Terjadi kesalahan tak terduga: {e}")
            # Tidak ada 'else' untuk tombol "Proses Notulen" di sini,
            # karena st.audio dan tombol itu sendiri sudah menunjukkan kondisi file diunggah.
        else:
            # Jika uploaded_file adalah None (kondisi awal, belum ada file diunggah)
            st.info("Unggah file audio untuk memulai proses.")

        st.markdown('</div>', unsafe_allow_html=True) # Tutup div upload-section

with right_col:
    with st.container():
        st.markdown('<div class="logo-box">', unsafe_allow_html=True)
        # Menggunakan Google Drive link langsung untuk gambar FTI Untar
        st.image("https://fti.untar.ac.id/wp-content/uploads/2023/12/Logo-Fakultas-TI-1.png", width=12, use_container_width=True)
        st.markdown('</div>', unsafe_allow_html=True)
        st.markdown("""
    Aplikasi NotulaX membantu Anda membuat transkripsi dan ringkasan otomatis dari rekaman audio rapat Anda. Ikuti langkah-langkah mudah ini:

    1.  **Unggah File Audio**:
        * Klik tombol "Browse files" atau seret file audio (WAV, MP3, OGG, FLAC) ke area unggah. Batas ukuran file 200MB.

    2.  **Periksa Audio (Opsional)**:
        * Pemutar audio akan muncul setelah unggah.

    3.  **Proses Notulen**:
        * Klik tombol "Proses Notulen". Harap tunggu, proses ini mungkin memakan waktu.

    4.  **Lihat Hasil**:
        * Transkripsi lengkap dan ringkasan akan ditampilkan.

    5.  **Unduh Notulen**:
        * Gunakan tombol unduh untuk menyimpan transkripsi atau ringkasan sebagai file teks.
    """, unsafe_allow_html=True)
        st.markdown("Paper: ")
        st.markdown("Contoh Audio File yang Dapat Digunakan: https://drive.google.com/drive/folders/1lveN4cF9of8OXqR6gCrm8glLvg3Pcvpw?usp=drive_link")



Overwriting main_frontend.py


In [6]:
# Jalankan Streamlit di background, redirect output ke file log
!nohup streamlit run main_frontend.py > streamlit_log.txt 2>&1 &

import time
time.sleep(5)

# Log Streamlit untuk memeriksa jika ada masalah
print("\nStreamlit log:")
!cat streamlit_log.txt


Streamlit log:



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.25.238:8501



In [7]:
!cat fastapi_backend.log

2025-06-17 15:15:28.557922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750173328.621545   53644 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750173328.653187   53644 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 15:15:28.766240: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
                                                                                                                     